<br>

# Introdução

Atualiza o valor da UFESP


In [ ]:
from datetime import datetime

import pandas as pd

import ufesp
from ufesp.paths import data_path

In [ ]:
# import ssl
# import sys
# from pathlib import Path

In [ ]:
ufesp.update_table()

In [ ]:
ufesp.get_table()

In [ ]:
# Com dia (string)
my_date_string = '2026-01-06'
dados = ufesp.get_ufesp_from_date(date=my_date_string)
dados['valor']

In [ ]:
# Com dia (datetime)
my_date = datetime.strptime(my_date_string, r'%Y-%m-%d')
dados = ufesp.get_ufesp_from_date(date=my_date)
dados['valor']

In [ ]:
# Com Ano
my_year = 2025
dados = ufesp.get_ufesp_from_year(year=my_year)
dados['valor']

<br>

---

## ddd


In [ ]:
# ssl._create_default_https_context = ssl._create_unverified_context

In [ ]:
url = 'https://legislacao.fazenda.sp.gov.br/Paginas/ValoresDaUFESP.aspx'
dfs = pd.read_html(io=url)
df = dfs[3]
df.head()

In [ ]:
df['VALOR EM R$'] = df['VALOR EM R$'].replace(',', '', regex=True)
df['VALOR EM R$'] = df['VALOR EM R$'].replace('\u200b', '', regex=True)
df['VALOR EM R$'] = df['VALOR EM R$'].astype(float)
df['VALOR EM R$'] = df['VALOR EM R$'] / 100

# Results
df.head()

In [ ]:
df['PERÍODO'] = df['PERÍODO'].replace('de ', '', regex=True)
df['PERÍODO'] = df['PERÍODO'].replace(' a ', ' A ', regex=True)

df['PERÍODO'] = df['PERÍODO'].replace('\u200b', '', regex=True)
df['PERÍODO'] = df['PERÍODO'].replace('\xa0', ' ', regex=True)

# Results
df.info()
df.head()

In [ ]:
df[['data_inicio', 'data_fim']] = df['PERÍODO'].str.split(
    pat='A',
    n=1,
    expand=True,
)

In [ ]:
# Ajusta Data do Início
df['data_inicio'] = df.loc[:, 'data_inicio'].str.strip()
df['data_inicio'] = pd.to_datetime(df['data_inicio'], format='%d/%m/%Y')


# Ajusta Data do Fim
df['data_fim'] = df.loc[:, 'data_fim'].str.strip()
df['data_fim'] = pd.to_datetime(df['data_fim'], format='%d/%m/%Y')

# Ajusta
df.loc[:, 'ano_mes'] = pd.to_datetime(df['data_inicio']).dt.to_period('M')

In [ ]:
# Renomeia Colunas
df = df.rename(
    mapper={
        'VALOR EM R$': 'valor',
        'BASE LEGAL': 'base_legal',
    },
    axis=1,
    inplace=False,
)

# 
df['base_legal'] = df['base_legal'].replace('\u200b', '', regex=True)
df['base_legal'] = df['base_legal'].replace('\xa0', ' ', regex=True)
df['base_legal'] = df['base_legal'].replace('88/ 25', '88/25', regex=True)
df['base_legal'] = df.loc[:, 'base_legal'].str.strip()

In [ ]:
# Results
df = df[
    [
        'data_inicio',
        'data_fim',
        'ano_mes',
        'valor',
        'base_legal',
    ]
].copy()


# Results
df.info()
df.head()

In [ ]:
# Save
df.to_csv(
    data_path / 'ufesp.csv',
    index=False,
    decimal=',',
)